In [1]:
import scipy.io
from numpy import linalg as LA
import numpy as np

mat = scipy.io.loadmat('pp_rcd_1.mat')
pp = mat['pp']

print(pp.shape)



(6660, 5)


In [3]:
layer = 18
num_p_real = 2*layer

def Ls(pp,pm,mp,mm,mat_index):
    
    n_entry = mat_index[:,0].size
    Hessian = np.zeros((num_p_real,num_p_real))
    
    for k in range(n_entry):
        i = index[k,0]
        j = index[k,1]
        Hessian[i,j] = np.real(pp[k]-pm[k]-mp[k]+mm[k])
        
    Hessian = Hessian + np.conjugate(np.transpose(Hessian)) - np.diag(np.diag(Hessian))
    
    e = LA.eigvals(Hessian)
    
    return np.max(np.absolute(e)), np.mean(np.absolute(np.diag(Hessian))), np.max(np.absolute(np.diag(Hessian)))

In [18]:



index = []
count = -1
for i in range(num_p_real):
    for j in range(i,num_p_real):
        count = count + 1 
        index.append([i,j,count])
        
index = np.asarray(index)


mat = scipy.io.loadmat('PP_RCD.mat')
PP = mat['PP']

mat = scipy.io.loadmat('PM_RCD.mat')
PM = mat['PM']

mat = scipy.io.loadmat('MP_RCD.mat')
MP = mat['MP']

mat = scipy.io.loadmat('MM_RCD.mat')
MM = mat['MM']

num_traj = 10
num_results = 132
num_entry = 666

L2_mat = np.empty((0,num_results), int)
Lavg_mat = np.empty((0,num_results), int)
Lmax_mat = np.empty((0,num_results), int)




for i in range(num_traj):
    l2 = []
    lavg = []
    lmax = []
    for j in range(num_results):
        res_1 = PP[num_entry*i:num_entry*(i+1),j]
        res_2 = PM[num_entry*i:num_entry*(i+1),j]
        res_3 = MP[num_entry*i:num_entry*(i+1),j]
        res_4 = MM[num_entry*i:num_entry*(i+1),j]
        L2, Lavg, Lmax = Ls(res_1,res_2,res_3,res_4,index)
        l2.append(L2)
        lavg.append(Lavg)
        lmax.append(Lmax)
    l2val = np.asarray([l2])
    L2_mat = np.append(L2_mat, l2val, axis = 0)
    lavgval = np.asarray([lavg])
    Lavg_mat = np.append(Lavg_mat, lavgval, axis = 0)
    lmaxval = np.asarray([lmax])
    Lmax_mat = np.append(Lmax_mat, lmaxval, axis = 0)

scipy.io.savemat('L2_rcd.mat', {'L2_rcd': L2_mat})
scipy.io.savemat('Lavg_rcd.mat', {'Lavg_rcd': Lavg_mat})
scipy.io.savemat('Lmax_rcd.mat', {'Lmax_rcd': Lmax_mat})